## Movie Recommendation System

In [13]:
from __future__ import print_function, division
from builtins import range, input

import pickle
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

### Data Preprocessing
+ Note: user ids are ordered sequentially from 1..138493 with no missing numbers, movie ids are integers from 1..131262 and NOT all movie ids appear, there are only 26744 movie ids

In [2]:
df = pd.read_csv('movielens-20m-dataset/rating.csv')

In [3]:
# make the user ids go from 0...N-1
df.userId = df.userId - 1

# create a mapping for movie ids
unique_movie_ids = set(df.movieId.values)
movie2idx = {}
count = 0
for movie_id in unique_movie_ids:
  movie2idx[movie_id] = count
  count += 1

# add them to the data frame
# takes awhile
df['movie_idx'] = df.apply(lambda row: movie2idx[row.movieId], axis=1)

df = df.drop(columns=['timestamp']) # we don't need timestamp data

In [4]:
df.to_csv('movielens-20m-dataset/edited_rating.csv', index=False)

### shrinking data 
+ full dataset is too large to perform O(n^2) algorithm 
+ select subset of users and moives, users who rated the most movies, movies who've been rated by the most users

In [7]:
df = pd.read_csv('movielens-20m-dataset/edited_rating.csv')
print("original dataframe size:", len(df))

original dataframe size: 20000263


In [8]:
N = df.userId.max() + 1 # number of users
M = df.movie_idx.max() + 1 # number of movies

user_ids_count = Counter(df.userId)
movie_ids_count = Counter(df.movie_idx)

# number of users and movies we would like to keep
n = 10000
m = 2000

user_ids = [u for u, c in user_ids_count.most_common(n)]
movie_ids = [m for m, c in movie_ids_count.most_common(m)]

# make a copy, otherwise ids won't be overwritten
df_small = df[df.userId.isin(user_ids) & df.movie_idx.isin(movie_ids)].copy()

df_small.head()

,userId,movieId,rating,movie_idx
960,10,1,4.5,1
961,10,10,2.5,10
962,10,19,3.5,19
963,10,32,5.0,32
964,10,39,4.5,39


In [9]:
# need to remake user ids and movie ids since they are no longer sequential
new_user_id_map = {}
i = 0
for old in user_ids:
  new_user_id_map[old] = i
  i += 1
print("i:", i)

new_movie_id_map = {}
j = 0
for old in movie_ids:
  new_movie_id_map[old] = j
  j += 1
print("j:", j)

i: 10000
j: 2000


In [10]:
print("Setting new ids")
df_small.loc[:, 'userId'] = df_small.apply(lambda row: new_user_id_map[row.userId], axis=1)
df_small.loc[:, 'movie_idx'] = df_small.apply(lambda row: new_movie_id_map[row.movie_idx], axis=1)
# df_small.drop(columns=['userId', 'movie_idx'])
# df_small.rename(index=str, columns={'new_userId': 'userId', 'new_movie_idx': 'movie_idx'})
print("max user id:", df_small.userId.max())
print("max movie id:", df_small.movie_idx.max())
print("small dataframe size:", len(df_small))

Setting new ids
max user id: 9999
max movie id: 1999


In [12]:
df_small.to_csv('movielens-20m-dataset/small_rating.csv', index=False)

### Table to Dictionary
+ user2movie: user ID -> movie ID
+ movie2user: movie ID -> user ID
+ usermovie2rating: (user ID, movie ID) -> rating 

In [16]:
df = pd.read_csv('movielens-20m-dataset/small_rating.csv')

N = df.userId.max() + 1 # number of users
M = df.movie_idx.max() + 1 # number of movies

# split into train and test
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

In [19]:
# a dictionary to tell us which users have rated which movies
user2movie = {}
# a dicationary to tell us which movies have been rated by which users
movie2user = {}
# a dictionary to look up ratings
usermovie2rating = {}
print("Calling: update_user2movie_and_movie2user")
count = 0
def update_user2movie_and_movie2user(row):
  global count
  count += 1
  if count % 100000 == 0:
    print("processed: %.3f" % (float(count)/cutoff))

  i = int(row.userId)
  j = int(row.movie_idx)
  if i not in user2movie:
    user2movie[i] = [j]
  else:
    user2movie[i].append(j)

  if j not in movie2user:
    movie2user[j] = [i]
  else:
    movie2user[j].append(i)

  usermovie2rating[(i,j)] = row.rating
df_train.apply(update_user2movie_and_movie2user, axis=1)

# test ratings dictionary
usermovie2rating_test = {}
print("Calling: update_usermovie2rating_test")
count = 0
def update_usermovie2rating_test(row):
  global count
  count += 1
  if count % 100000 == 0:
    print("processed: %.3f" % (float(count)/len(df_test)))

  i = int(row.userId)
  j = int(row.movie_idx)
  usermovie2rating_test[(i,j)] = row.rating
df_test.apply(update_usermovie2rating_test, axis=1)


Calling: update_user2movie_and_movie2user
processed: 0.023
processed: 0.046
processed: 0.070
processed: 0.093
processed: 0.116
processed: 0.139
processed: 0.162
processed: 0.185
processed: 0.209
processed: 0.232
processed: 0.255
processed: 0.278
processed: 0.301
processed: 0.325
processed: 0.348
processed: 0.371
processed: 0.394
processed: 0.417
processed: 0.440
processed: 0.464
processed: 0.487
processed: 0.510
processed: 0.533
processed: 0.556
processed: 0.580
processed: 0.603
processed: 0.626
processed: 0.649
processed: 0.672
processed: 0.695
processed: 0.719
processed: 0.742
processed: 0.765
processed: 0.788
processed: 0.811
processed: 0.835
processed: 0.858
processed: 0.881
processed: 0.904
processed: 0.927
processed: 0.950
processed: 0.974
processed: 0.997
Calling: update_usermovie2rating_test
processed: 0.093
processed: 0.185
processed: 0.278
processed: 0.371
processed: 0.464
processed: 0.556
processed: 0.649
processed: 0.742
processed: 0.835
processed: 0.927


1761954    None
22147      None
4476511    None
3719981    None
4884866    None
1886693    None
2010095    None
1534684    None
1872587    None
2697224    None
3779918    None
3087537    None
1599772    None
4618985    None
2142242    None
354729     None
2168094    None
1705211    None
2509466    None
4355676    None
922175     None
278472     None
1423300    None
4844620    None
1423051    None
547157     None
1356751    None
1910798    None
3337704    None
4373420    None
           ... 
2830127    None
1650971    None
1457496    None
4713814    None
1431263    None
5333704    None
3135237    None
3009113    None
4120737    None
1667744    None
2422003    None
2283902    None
4211       None
3474391    None
4380524    None
2453923    None
1884382    None
2432942    None
279290     None
812743     None
1463157    None
4592163    None
900890     None
4187977    None
246352     None
1882367    None
3127037    None
3347540    None
1701404    None
63381      None
Length: 1078405, dtype: 

In [20]:
# note: these are not really JSONs
with open('user2movie.json', 'wb') as f:
  pickle.dump(user2movie, f)

with open('movie2user.json', 'wb') as f:
  pickle.dump(movie2user, f)

with open('usermovie2rating.json', 'wb') as f:
  pickle.dump(usermovie2rating, f)

with open('usermovie2rating_test.json', 'wb') as f:
  pickle.dump(usermovie2rating_test, f)

### User-based Collaborative Filtering 